# Finance Data OS — Week 5 Notebook

**Theme:** Costs, Controls & Tuning

This notebook:
1) Loads `lake/feature_mart.parquet`
2) Builds **signals_mart_v2** from params (`sma_fast`, `sma_slow`, `vol_threshold`)
3) Runs a **long-only backtest** with **cost_bps** (next-bar execution)
4) Logs **trades** & computes **KPIs** (after cost)
5) Optionally runs a **small grid search** and writes `tuning_mart.parquet`
6) Writes Parquet outputs for Power BI:
   - `lake/signals_mart_v2.parquet`
   - `lake/backtest_mart_v2/<TICKER>.parquet`
   - `lake/backtest_mart_v2/_summary.parquet`
   - `lake/trades_mart/<TICKER>.parquet`
   - `lake/tuning_mart.parquet` (optional)

> If paths fail, check the “Path Discovery” cell and confirm the `lake` folder exists at the repo root.


# Imports & Path Discovery


In [1]:
# --- Robust repo root / lake discovery (Windows-safe & cross-platform) ---
from pathlib import Path

def find_lake_root():
    here = Path.cwd()
    candidates = [here, *here.parents]          # walk up: . .. ../.. ../... etc.
    lake_dirs = [c / "lake" for c in candidates if (c / "lake").exists()]

    if not lake_dirs:
        raise FileNotFoundError(
            "Could not find a 'lake' folder in this directory or any parent.\n"
            "Open the notebook from inside your repo, e.g. .../Finance Data OS/notebooks"
        )

    # Prefer the candidate that actually contains our Week 3/4 parquet files
    # If none contain files, fall back to the nearest 'lake' and warn.
    def has_expected_files(p: Path) -> bool:
        return any((p / fname).exists() for fname in [
            "feature_mart.parquet",
            "signals_mart.parquet",
            "_summary.parquet"
        ]) or (p / "backtest_mart").exists()

    for ld in lake_dirs:
        if has_expected_files(ld):
            return ld

    # Fallback to the closest lake, but warn loudly
    ld = lake_dirs[0]
    print("[WARN] Using a 'lake' folder that does not contain expected files:", ld)
    return ld

LAKE_ROOT = find_lake_root()

# Define important paths (do NOT create LAKE_ROOT here)
FEATURE_MART = LAKE_ROOT / "feature_mart.parquet"
SIGNALS_MART = LAKE_ROOT / "signals_mart.parquet"
BACKTEST_DIR = LAKE_ROOT / "backtest_mart_v2"   # Week 5 outputs (ok to create later)
SUMMARY_FILE = BACKTEST_DIR / "_summary.parquet"

print("CWD       :", Path.cwd())
print("LAKE_ROOT :", LAKE_ROOT.resolve())
print("FEATURE   :", FEATURE_MART.resolve())
print("SIGNALS   :", SIGNALS_MART.resolve())
print("BACKTEST  :", BACKTEST_DIR.resolve())
print("SUMMARY   :", SUMMARY_FILE.resolve())

# Sanity: prevent accidental notebooks\lake
nb_lake = Path.cwd() / "lake"
if nb_lake.exists() and nb_lake.resolve() != LAKE_ROOT.resolve():
    raise RuntimeError(
        f"Found a 'lake' under notebooks: {nb_lake}\n"
        "Delete that folder (it was created by mistake) and re-run the cell."
    )


CWD       : C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\notebooks
LAKE_ROOT : C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake
FEATURE   : C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake\feature_mart.parquet
SIGNALS   : C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake\signals_mart.parquet
BACKTEST  : C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake\backtest_mart_v2
SUMMARY   : C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake\backtest_mart_v2\_summary.parquet


# Parameters and Helpers


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

# Tunables
SMA_FAST = 25
SMA_SLOW = 100
VOL_WINDOW = 20          # stdev of return1 (in % space if that’s how you built it)
VOL_THRESHOLD = 20.0     # “high vol” threshold (%)
COST_BPS = 5             # 5 bps each trade
COOLDOWN_DAYS = 0        # optional day(s) after exit before re-enter

def sma(series: pd.Series, n: int) -> pd.Series:
    return series.rolling(n, min_periods=max(5, int(n*0.3))).mean()

def build_signals_v2(df: pd.DataFrame,
                     sma_fast=SMA_FAST,
                     sma_slow=SMA_SLOW,
                     vol_threshold=VOL_THRESHOLD) -> pd.DataFrame:
    g = df.sort_values(["ticker","date"]).copy()
    g["sma_fast"] = g.groupby("ticker")["close"].transform(lambda s: sma(s, sma_fast))
    g["sma_slow"] = g.groupby("ticker")["close"].transform(lambda s: sma(s, sma_slow))

    # assume vol20 already exists as daily % stdev in your mart; otherwise compute here
    vol_col = "vol20"
    if vol_col not in g.columns:
        # fallback: stdev of daily % returns
        g["ret1"] = g.groupby("ticker")["close"].pct_change()
        g[vol_col] = g.groupby("ticker")["ret1"].transform(lambda s: s.rolling(VOL_WINDOW).std()*100)

    g["high_vol"] = g[vol_col] >= vol_threshold
    g["long_rule"] = (g["sma_fast"] >= g["sma_slow"]) & (~g["high_vol"])
    g["exit_rule"] = (g["sma_fast"] <  g["sma_slow"]) | (g["high_vol"])
    return g

def run_backtest_with_costs(signals: pd.DataFrame,
                            cost_bps=COST_BPS,
                            cooldown=COOLDOWN_DAYS) -> pd.DataFrame:
    """Next-bar execution, long-only, costs on entry/exit."""
    g = signals.sort_values(["ticker","date"]).copy()
    # daily % return assumed in 'return1' (e.g., 0.25% => 0.0025). If your mart uses 'return1'
    r = g["return1"].astype(float)

    # position: hold next day after today's long rule
    pos = g["long_rule"].shift(1).fillna(False).astype(int)

    if cooldown > 0:
        # lockout period after exit: when exit true, force next N days to 0
        ex = g["exit_rule"].shift(1).fillna(False)
        lock = ex.groupby(g["ticker"]).apply(lambda s: s.replace({True: cooldown})\
                                             .groupby(level=0).cumsum()).reset_index(level=0, drop=True)
        pos = np.where(lock>0, 0, pos)

    # trades: change in position
    trade = pos - pos.shift(1).fillna(0)
    # cost in return units (bps -> fraction)
    cost = np.abs(trade) * (cost_bps / 10_000.0)
    ret_after_cost = pos * r - cost

    out = g[["ticker","date","close"]].copy()
    out["position"] = pos
    out["ret_after_cost"] = ret_after_cost
    out["equity"] = (1 + ret_after_cost).groupby(out["ticker"]).cumprod()
    return out

def kpi(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for tkr, d in df.groupby("ticker"):
        r = d["ret_after_cost"].dropna()
        cagr = (1+r).prod()**(252/len(r)) - 1 if len(r) else np.nan
        sharpe = (r.mean()/r.std()*np.sqrt(252)) if r.std()>0 else np.nan
        dd = d["equity"]/d["equity"].cummax() - 1
        maxdd = float(dd.min()) if len(dd) else np.nan
        rows.append({"ticker": tkr, "CAGR %": round(cagr*100,2),
                     "Sharpe": round(sharpe,2) if np.isfinite(sharpe) else np.nan,
                     "Max DD %": round(100*maxdd,2) if np.isfinite(maxdd) else np.nan})
    return pd.DataFrame(rows).sort_values("ticker")


# Build Signals_v2 & Run Backtest (Python)


In [3]:

fm = pd.read_parquet(FEATURE_MART)
fm["date"] = pd.to_datetime(fm["date"])


sig = build_signals_v2(fm)
bt  = run_backtest_with_costs(sig)


bt_zero = run_backtest_with_costs(sig, cost_bps=0)
def single_kpis(df):  # for a single ticker (e.g., AAPL)
    r = df["ret_after_cost"].dropna()
    cagr = (1+r).prod()**(252/len(r)) - 1 if len(r) else np.nan
    sharpe = (r.mean()/r.std()*np.sqrt(252)) if r.std()>0 else np.nan
    return cagr, sharpe

tkr = "AAPL"
cagr_cur, sharpe_cur = single_kpis(bt.query("ticker == @tkr"))
cagr_zer, sharpe_zer = single_kpis(bt_zero.query("ticker == @tkr"))
print(f"CAGR% current vs zero-cost: {cagr_cur*100:0.2f}  vs  {cagr_zer*100:0.2f}")
print(f"Sharpe current vs zero-cost: {sharpe_cur:0.2f}  vs  {sharpe_zer:0.2f}")


BACKTEST_DIR.mkdir(parents=True, exist_ok=True)
for tkr, d in bt.groupby("ticker"):
    (BACKTEST_DIR / f"{tkr}.parquet").unlink(missing_ok=True)
    d.to_parquet(BACKTEST_DIR / f"{tkr}.parquet", index=False)

kpi_df = kpi(bt)
kpi_df.to_parquet(SUMMARY_FILE, index=False)
kpi_df


CAGR% current vs zero-cost: 19.14  vs  19.32
Sharpe current vs zero-cost: 0.81  vs  0.82


C:\Users\TJs PC\AppData\Local\Temp\ipykernel_11044\584814432.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pos = g["long_rule"].shift(1).fillna(False).astype(int)
C:\Users\TJs PC\AppData\Local\Temp\ipykernel_11044\584814432.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pos = g["long_rule"].shift(1).fillna(False).astype(int)


,ticker,CAGR %,Sharpe,Max DD %
0,AAPL,19.14,0.81,-31.43
1,MSFT,18.30,0.84,-28.90
2,NVDA,57.27,1.26,-37.99
3,TSLA,40.20,0.94,-64.85


# Trades Log (Python)


In [4]:
# --- Trades helpers (define once) ---
import pandas as pd
import numpy as np
from pathlib import Path

def load_per_ticker_daily() -> pd.DataFrame:
    """
    Return a DataFrame with date, ticker, return1, position.
    Uses in-memory 'per_ticker' if it exists; otherwise reads parquet files
    under lake/backtest_mart (skips _summary.parquet).
    Robust to column-name variants (return/ret1/return1, position/long_rule_i).
    """
    try:
        g = per_ticker     # may not exist on a fresh kernel
        low = {c.lower(): c for c in g.columns}
        need = ["date","ticker","return1","position"]
        if not all(n in low for n in need):
            raise KeyError("per_ticker missing required columns")
        out = g[[low[n] for n in need]].copy()
    except Exception:
        # Fallback: read files from backtest_mart
        LAKE_ROOT   = next(c for c in [Path.cwd(), *Path.cwd().parents] if (c / "lake").exists()) / "lake"
        BACKTEST_DIR = LAKE_ROOT / "backtest_mart"
        files = sorted(p for p in BACKTEST_DIR.glob("*.parquet") if not p.name.startswith("_"))
        if not files:
            raise FileNotFoundError(f"No per-ticker parquet files found in {BACKTEST_DIR}. Run the backtest cell first.")
        parts = []
        for p in files:
            df  = pd.read_parquet(p)
            low = {c.lower(): c for c in df.columns}
            col_date = low.get("date")
            col_tkr  = low.get("ticker")
            col_pos  = low.get("position") or low.get("long_rule_i")
            col_ret1 = low.get("return1") or low.get("ret1") or low.get("ret")
            missing = [k for k,v in dict(date=col_date,ticker=col_tkr,return1=col_ret1,position=col_pos).items() if v is None]
            if missing:
                # just skip non-daily files like _summary
                continue
            d = df[[col_date,col_tkr,col_ret1,col_pos]].copy()
            d.columns = ["date","ticker","return1","position"]
            parts.append(d)
        if not parts:
            raise RuntimeError(f"Found parquet files in {BACKTEST_DIR} but none had the required "
                               "columns (date, ticker, return1, position).")
        out = pd.concat(parts, ignore_index=True)

    out["date"]     = pd.to_datetime(out["date"])
    out["position"] = out["position"].astype(int)
    return out.sort_values(["ticker","date"]).reset_index(drop=True)

def trades_from_position(df: pd.DataFrame, cost_bps: int = 0) -> pd.DataFrame:
    """
    Build trade records from a 0/1 position series.
    Cost model: charge cost on both entry and exit (bps per side).
    """
    d = df.sort_values(["ticker","date"]).copy()
    # change in position → trade (+1 buy, -1 sell)
    d["trade"] = d.groupby("ticker")["position"].diff().fillna(d["position"])
    trades = d.loc[d["trade"] != 0, ["date","ticker","return1","position","trade"]].copy()
    # simple costs (entry + exit)
    trades["cost_pct"] = (2 * cost_bps) / 10_000.0
    trades["pnl_pct"]  = trades["position"] * trades["return1"]
    trades["pnl_after_cost_pct"] = (trades["pnl_pct"] - trades["cost_pct"]).round(4)
    return trades


In [5]:
from pathlib import Path
import pandas as pd
import numpy as np

def find_lake_root() -> Path:
    here = Path.cwd()
    for cand in [here, *here.parents]:
        if (cand / "lake").exists():
            return cand / "lake"
    raise FileNotFoundError("Couldn't find a 'lake' folder in this directory or any parent.")

LAKE_ROOT   = find_lake_root()
BACKTEST_DIR = LAKE_ROOT / "backtest_mart"
TRADES_DIR   = LAKE_ROOT / "trades_mart_v1"
COST_BPS     = globals().get("COST_BPS", 5)  # tweak if you like; 5 bps = 0.05%
print("[OK] LAKE_ROOT ->", LAKE_ROOT.resolve())


[OK] LAKE_ROOT -> C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake


In [6]:
per_ticker_daily = load_per_ticker_daily()
print("[OK] per_ticker_daily:", per_ticker_daily.shape, "columns:", list(per_ticker_daily.columns))

trades = trades_from_position(per_ticker_daily, cost_bps=COST_BPS)
print("[OK] trades:", trades.shape)

TRADES_DIR.mkdir(parents=True, exist_ok=True)
for tkr, d in trades.groupby("ticker"):
    d.to_parquet(TRADES_DIR / f"{tkr}.parquet", index=False)
print("[OK] trades saved ->", TRADES_DIR.resolve())

display(trades.head())


[OK] per_ticker_daily: (6184, 4) columns: ['date', 'ticker', 'return1', 'position']
[OK] trades: (181, 8)
[OK] trades saved -> C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake\trades_mart_v1


,date,ticker,return1,position,trade,cost_pct,pnl_pct,pnl_after_cost_pct
15,2019-02-21,AAPL,-0.005639,1,1.0,0.001,-0.005639,-0.0066
77,2019-05-21,AAPL,0.019171,0,-1.0,0.001,0.000000,-0.0010
99,2019-06-21,AAPL,-0.003409,1,1.0,0.001,-0.003409,-0.0044
259,2020-03-09,AAPL,-0.079092,0,-1.0,0.001,-0.000000,-0.0010
291,2020-04-23,AAPL,-0.003876,1,1.0,0.001,-0.003876,-0.0049


# Grid Tuning (Python)


In [7]:
# --- Grid Tuning (clean version) ---
from pathlib import Path
import pandas as pd

# Locate lake and set paths (same logic you already use)
REPO_ROOT  = next(c for c in [Path.cwd(), *Path.cwd().parents] if (c / "lake").exists())
LAKE_ROOT  = REPO_ROOT / "lake"
FEATURE_MART = LAKE_ROOT / "feature_mart.parquet"
TUNING_PATH  = LAKE_ROOT / "tuning_mart.parquet"

# Grids (edit if you like)
GRID_FAST = [10, 15, 20, 30]
GRID_SLOW = [50, 100, 150, 200]
GRID_VOL  = [15.0, 20.0, 25.0]     # high-vol threshold (%)
COST_BPS  = 5

# Load feature mart and build once
fm = pd.read_parquet(FEATURE_MART)
fm["date"] = pd.to_datetime(fm["date"])
fm = fm.sort_values(["ticker","date"]).reset_index(drop=True)

rows = []
for f in GRID_FAST:
    for s in GRID_SLOW:
        for v in GRID_VOL:
            sig   = build_signals_v2(fm, f, s, v)
            daily = run_backtest_with_costs(sig, cost_bps=COST_BPS)  # returns one DataFrame
            summ  = kpi(daily)                                       # use your KPI() to summarize
            if not summ.empty:
                summ["sma_fast"]      = f
                summ["sma_slow"]      = s
                summ["vol_threshold"] = v
                rows.append(summ)

tune = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()
if not tune.empty:
    tune.to_parquet(TUNING_PATH, index=False)
    print(f"[OK] tuning_mart -> {TUNING_PATH.resolve()}")
    display(tune.head())
else:
    print("[WARN] tuning grid produced no rows (check inputs).")


C:\Users\TJs PC\AppData\Local\Temp\ipykernel_11044\584814432.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pos = g["long_rule"].shift(1).fillna(False).astype(int)
C:\Users\TJs PC\AppData\Local\Temp\ipykernel_11044\584814432.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pos = g["long_rule"].shift(1).fillna(False).astype(int)
C:\Users\TJs PC\AppData\Local\Temp\ipykernel_11044\584814432.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(c

[OK] tuning_mart -> C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake\tuning_mart.parquet


,ticker,CAGR %,Sharpe,Max DD %,sma_fast,sma_slow,vol_threshold
0,AAPL,24.05,1.13,-25.98,10,50,15.0
1,MSFT,10.95,0.62,-24.21,10,50,15.0
2,NVDA,42.35,1.07,-39.80,10,50,15.0
3,TSLA,27.49,0.82,-56.90,10,50,15.0
4,AAPL,24.05,1.13,-25.98,10,50,20.0


In [8]:
# --- Grid Tuning (clean, robust) ---
from pathlib import Path
import pandas as pd

# Locate lake (same logic you already use above)
REPO_ROOT = next(c for c in [Path.cwd(), *Path.cwd().parents] if (c / "lake").exists())
LAKE_ROOT  = REPO_ROOT / "lake"

FEATURE_MART = LAKE_ROOT / "feature_mart.parquet"
TUNING_PATH  = LAKE_ROOT / "tuning_mart.parquet"

# Grids (edit if you like)
GRID_FAST = [10, 15, 20, 30]
GRID_SLOW = [50, 100, 150, 200]
GRID_VOL  = [15.0, 20.0, 25.0]   # high-vol threshold (%)
COST_BPS  = 5                    # per side

# Load the feature mart
fm = pd.read_parquet(FEATURE_MART)
fm["date"] = pd.to_datetime(fm["date"])
fm = fm.sort_values(["ticker","date"]).reset_index(drop=True)

def run_grid_tuning(fm: pd.DataFrame,
                    grid_fast: list[int],
                    grid_slow: list[int],
                    grid_vol:  list[float],
                    cost_bps:  int) -> pd.DataFrame:
    """Run the grid search; returns one row per (ticker, param combo) with KPIs."""
    rows: list[pd.DataFrame] = []

    for f in grid_fast:
        for s in grid_slow:
            for v in grid_vol:
                # build signals for this combo (uses your helpers defined above)
                sig   = build_signals_v2(fm, f, s, v)
                daily = run_backtest_with_costs(sig, cost_bps=cost_bps)  # returns per-day df
                summ  = kpi(daily)                                      # summarize per ticker

                if not summ.empty:
                    summ = summ.copy()
                    summ["sma_fast"]      = f
                    summ["sma_slow"]      = s
                    summ["vol_threshold"] = v
                    rows.append(summ)

    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()

# Run, save, and preview
tune = run_grid_tuning(fm, GRID_FAST, GRID_SLOW, GRID_VOL, COST_BPS)

if not tune.empty:
    tune.to_parquet(TUNING_PATH, index=False)
    print(f"[OK] tuning_mart -> {TUNING_PATH.resolve()}")
    display(tune.head())
else:
    print("[WARN] tuning grid produced no rows (check inputs).")


C:\Users\TJs PC\AppData\Local\Temp\ipykernel_11044\584814432.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pos = g["long_rule"].shift(1).fillna(False).astype(int)
C:\Users\TJs PC\AppData\Local\Temp\ipykernel_11044\584814432.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pos = g["long_rule"].shift(1).fillna(False).astype(int)
C:\Users\TJs PC\AppData\Local\Temp\ipykernel_11044\584814432.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(c

[OK] tuning_mart -> C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake\tuning_mart.parquet


C:\Users\TJs PC\AppData\Local\Temp\ipykernel_11044\584814432.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pos = g["long_rule"].shift(1).fillna(False).astype(int)
C:\Users\TJs PC\AppData\Local\Temp\ipykernel_11044\584814432.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pos = g["long_rule"].shift(1).fillna(False).astype(int)
C:\Users\TJs PC\AppData\Local\Temp\ipykernel_11044\584814432.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(c

,ticker,CAGR %,Sharpe,Max DD %,sma_fast,sma_slow,vol_threshold
0,AAPL,24.05,1.13,-25.98,10,50,15.0
1,MSFT,10.95,0.62,-24.21,10,50,15.0
2,NVDA,42.35,1.07,-39.80,10,50,15.0
3,TSLA,27.49,0.82,-56.90,10,50,15.0
4,AAPL,24.05,1.13,-25.98,10,50,20.0


# Power BI Hook 


## Power BI — new sources to add (absolute Windows paths)

- `lake/signals_mart_v2.parquet`
- `lake/backtest_mart_v2/_summary.parquet`  *(KPIs after cost)*
- Folder: `lake/backtest_mart_v2/` *(per-ticker dailies with equity_after_cost)*
- Folder: `lake/trades_mart/` *(per-trade logs)*
- `lake/tuning_mart.parquet` *(grid results — for heatmap/matrix)*

**New pages**
- **Costs & Sensitivity:** slicer for **COST_BPS**, cards (CAGR, Sharpe, Max DD, Win rate), line of **equity_after_cost**.
- **Parameter Tuning:** matrix/heatmap `sma_fast × sma_slow` colored by **Sharpe** (diverging, center at median).


# Sanity Check


In [11]:
# === Week 5 — One-shot Sanity Check (safe to run anytime) ===
from pathlib import Path
import numpy as np
import pandas as pd

# ---------- Locate lake ----------
def find_lake_root() -> Path:
    here = Path.cwd()
    cands = [p / "lake" for p in [here, *here.parents] if (p / "lake").exists()]
    if not cands:
        raise FileNotFoundError("Couldn't find a 'lake' folder near this notebook.")
    def has_expected(p: Path) -> bool:
        try:
            return any([
                (p / "feature_mart.parquet").exists(),
                (p / "signals_mart.parquet").exists(),
                (p / "signals_mart_v2.parquet").exists(),
                any(x.is_dir() and x.name.startswith("backtest_mart") for x in p.iterdir()),
            ])
        except Exception:
            return False
    # Prefer a lake that actually holds our Week 3/4/5 artifacts
    for p in cands:
        if has_expected(p):
            return p
    return cands[0]

LAKE_ROOT = find_lake_root()

# ---------- Key paths (robust to v1/v2 names) ----------
FEATURE_MART = LAKE_ROOT / "feature_mart.parquet"

SIGNALS_MART = LAKE_ROOT / "signals_mart.parquet"
if not SIGNALS_MART.exists():
    alt = LAKE_ROOT / "signals_mart_v2.parquet"
    if alt.exists(): SIGNALS_MART = alt

def pick_backtest_dir(lake: Path) -> Path:
    cands = [p for p in lake.glob("backtest_mart*") if p.is_dir()]
    if not cands:
        raise FileNotFoundError(f"No backtest_mart* folder under {lake}")
    p_v2 = lake / "backtest_mart_v2"
    return p_v2 if p_v2.exists() else max(cands, key=lambda p: p.stat().st_mtime)

BACKTEST_DIR = pick_backtest_dir(LAKE_ROOT)
SUMMARY_FILE = BACKTEST_DIR / "_summary.parquet"

TRADES_DIR  = LAKE_ROOT / "trades_mart"
TUNING_PATH = LAKE_ROOT / "tuning_mart.parquet"   # optional

# ---------- Pretty print the paths you’ll use in Power BI ----------
print("📁 LAKE_ROOT :", LAKE_ROOT.resolve())
print("📄 Feature   :", FEATURE_MART.resolve())
print("📄 Signals   :", (SIGNALS_MART.resolve() if SIGNALS_MART.exists() else "— (missing / optional)"))
print("📂 Backtest  :", BACKTEST_DIR.resolve(), "(per-ticker dailies)")
print("📄 Summary   :", SUMMARY_FILE.resolve())
print("📂 TradesDir :", (TRADES_DIR.resolve() if TRADES_DIR.exists() else "— (optional)"))
print("📄 Tuning    :", (TUNING_PATH.resolve() if TUNING_PATH.exists() else "— (optional)"))
print()

# ---------- Helper: compute KPIs from a per-ticker daily frame ----------
def compute_kpis(df: pd.DataFrame) -> tuple[float, float, float]:
    low = {c.lower(): c for c in df.columns}
    c_ret = low.get("ret_after_cost") or low.get("ret_strat") or low.get("return1")
    if c_ret is None:
        raise KeyError("Need a return column: ret_after_cost / ret_strat / return1")
    r = df[c_ret].astype(float).dropna()
    if len(r) == 0:
        return np.nan, np.nan, np.nan
    cagr   = (1.0 + r).prod()**(252.0/len(r)) - 1.0
    std    = r.std()
    sharpe = (r.mean()/std*np.sqrt(252.0)) if std > 0 else np.nan
    c_eq   = low.get("equity")
    if c_eq:
        eq = df[c_eq].astype(float)
    else:
        eq = (1.0 + r).cumprod()
    dd  = eq/eq.cummax() - 1.0
    mdd = float(dd.min()) if len(dd) else np.nan
    return float(cagr), float(sharpe), float(mdd)

# ---------- 1) Feature mart basics ----------
assert FEATURE_MART.exists(), f"Missing {FEATURE_MART}"
fm = pd.read_parquet(FEATURE_MART)
need_fm = {"date","ticker","close"}
assert need_fm.issubset({c.lower() for c in fm.columns}), f"feature_mart missing columns {need_fm}"
fm["date"] = pd.to_datetime(fm["date"])
assert fm["date"].notna().all(), "feature_mart: 'date' contains NaT"
print(f"✅ feature_mart: rows={len(fm):,}, tickers={fm['ticker'].nunique()}, window=[{fm['date'].min().date()} .. {fm['date'].max().date()}]")

# ---------- 2) Backtest folder & one sample file ----------
bt_files = [p for p in BACKTEST_DIR.glob("*.parquet") if not p.name.startswith("_")]
assert bt_files, f"No per-ticker parquet files found in {BACKTEST_DIR}"
sample = bt_files[0]
d0 = pd.read_parquet(sample)
low0 = {c.lower(): c for c in d0.columns}
for req in ["date","ticker"]:
    assert req in low0, f"{sample.name} missing '{req}'"
d0[low0["date"]] = pd.to_datetime(d0[low0["date"]])
assert d0[low0["date"]].notna().all(), f"{sample.name}: 'date' contains NaT"
# Position (if present) must be 0/1
c_pos = low0.get("position") or low0.get("long_rule_i")
if c_pos:
    pos_vals = set(pd.Series(d0[c_pos]).dropna().unique().tolist())
    assert pos_vals.issubset({0,1}), f"{sample.name}: position has non 0/1 values: {pos_vals}"
print(f"✅ backtest dailies: files={len(bt_files)}, sample={sample.name}")

# ---------- 3) KPI cross-check against summary ----------
assert SUMMARY_FILE.exists(), f"Missing {SUMMARY_FILE}"
summary = pd.read_parquet(SUMMARY_FILE)
need_kpi = {"ticker","CAGR %","Sharpe","Max DD %"}
assert need_kpi.issubset(set(summary.columns)), f"summary missing columns {need_kpi}"

tkr = str(d0[low0["ticker"]].iloc[0])
row = summary.loc[summary["ticker"] == tkr]
assert len(row) == 1, f"{tkr} not found uniquely in summary"
row = row.iloc[0]

cagr, sharpe, mdd = compute_kpis(d0)

# tolerances
CAGR_TOL_PP   = 0.5   # percentage points
SHARPE_TOL    = 0.05
MAXDD_TOL_PP  = 0.5

print(f"Compare KPIs for {tkr} (calc vs summary):")
print(f"  CAGR%  {cagr*100:0.2f}  vs  {float(row['CAGR %']):0.2f}")
print(f"  Sharpe {sharpe:0.2f}    vs  {float(row['Sharpe']):0.2f}")
print(f"  MaxDD% {mdd*100:0.2f}  vs  {float(row['Max DD %']):0.2f}")

ok_cagr   = np.isfinite(cagr)   and abs(cagr*100  - float(row["CAGR %"]))  <= CAGR_TOL_PP
ok_sharpe = np.isfinite(sharpe) and abs(sharpe    - float(row["Sharpe"]))  <= SHARPE_TOL
ok_mdd    = np.isfinite(mdd)    and abs(mdd*100   - float(row["Max DD %"])) <= MAXDD_TOL_PP

assert ok_cagr,   f"CAGR mismatch for {tkr}"
assert ok_sharpe, f"Sharpe mismatch for {tkr}"
assert ok_mdd,    f"Max DD mismatch for {tkr}"
print("✅ KPI compare passed")

# ---------- 4) Optional: trades & tuning quick peeks (no fail) ----------
if TRADES_DIR.exists():
    some_trade = next(iter(TRADES_DIR.glob("*.parquet")), None)
    if some_trade: 
        td = pd.read_parquet(some_trade)
        print(f"ℹ️ trades_mart present: {len(list(TRADES_DIR.glob('*.parquet')))} files, sample='{some_trade.name}' rows={len(td):,}")
if TUNING_PATH.exists():
    tp = pd.read_parquet(TUNING_PATH)
    need_tune = {"ticker","CAGR %","Sharpe","Max DD %","sma_fast","sma_slow","vol_threshold"}
    missing = need_tune.difference(set(tp.columns))
    if missing:
        print(f"ℹ️ tuning_mart present but missing columns {missing} (ok if you didn’t run grid yet)")
    else:
        print(f"ℹ️ tuning_mart present: rows={len(tp):,}, tickers={tp['ticker'].nunique()}")

print("\n🎉 All sanity checks passed.")


📁 LAKE_ROOT : C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake
📄 Feature   : C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake\feature_mart.parquet
📄 Signals   : C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake\signals_mart.parquet
📂 Backtest  : C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake\backtest_mart_v2 (per-ticker dailies)
📄 Summary   : C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake\backtest_mart_v2\_summary.parquet
📂 TradesDir : C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake\trades_mart
📄 Tuning    : C:\Users\TJs PC\OneDrive\Desktop\Finance Data OS\lake\tuning_mart.parquet

✅ feature_mart: rows=6,184, tickers=4, window=[2019-01-30 .. 2025-09-05]
✅ backtest dailies: files=4, sample=AAPL.parquet
Compare KPIs for AAPL (calc vs summary):
  CAGR%  19.14  vs  19.14
  Sharpe 0.81    vs  0.81
  MaxDD% -31.43  vs  -31.43
✅ KPI compare passed
ℹ️ trades_mart present: 4 files, sample='AAPL.parquet' rows=10
ℹ️ tuning_mart present: rows=192, tickers=4

🎉 All